In [8]:
import os
import numpy as np
import pickle

from main_as_fun import main_as_fun


is_array_job=False

if is_array_job:
    job_idx = int(os.environ["SLURM_ARRAY_TASK_ID"]) - 1
else:
    job_idx = 3


## Make folder to save best params
best_param_folder = '/home/erussek/projects/RNNs/best_hyper_params'

# Read in best hyper params...
best_param_dict = pickle.load(open(os.path.join(best_param_folder, 'best_hyper_params.pkl'), 'rb'))

# types of models...
model_names = ['LSTM', 'GRU', 'Transformer']

train_names = ['train_sim_test_sim', 'train_sim_test_human', 'train_human_test_human', 'train_sim_human_test_human']


this_train_name = train_names[0]

In [20]:
job_model_names = []
job_runs = []
job_hidden_sizes = []
job_sim_lrs = []
job_human_lrs = []
job_attention_heads = []
job_layers = []
job_runs = []
job_n_sim_seqs = []
job_n_human_seqs = []

n_runs = 10
for run in range(n_runs):
    for tn in train_names:
        for model_name in model_names:
            mbp = best_param_dict[tn][model_name]

            job_runs.append(run)
            job_model_names.append(model_name)
            job_hidden_sizes.append(mbp['d_model'])
            job_sim_lrs.append(mbp['sim_lr'])
            job_human_lrs.append(mbp['human_lr'])
            job_attention_heads.append(mbp['n_head'])
            job_layers.append(mbp['n_layers'])
            job_n_sim_seqs.append(mbp['best_sim_num'])
            job_n_human_seqs.append(mbp['best_hum_num'])


In [21]:
len(job_runs) # 120

120

In [15]:
# Add the train on human and fine-tune on sim jobs... 
# what's the file name
run_idx = job_runs[job_idx]
model_name = job_model_names[job_idx]
d_model = job_hidden_sizes[job_idx]
sim_lr = job_sim_lrs[job_idx]
human_lr = job_human_lrs[job_idx]
n_head = job_attention_heads[job_idx]
n_layers = job_layers[job_idx]
n_simulation_sequences_train = job_n_sim_seqs[job_idx]
n_human_sequences_train = job_n_human_seqs[job_idx]

save_file_name = 'run_{}_model_name_{}_fix_and_sim_bp'.format(run_idx, model_name)

# Call key function  -- 5e5 sequences?
main_as_fun(
    run_idx = job_runs[job_idx],
    model_name = job_model_names[job_idx],
    d_model = job_hidden_sizes[job_idx],
    sim_lr = job_sim_lrs[job_idx],
    human_lr = job_human_lrs[job_idx],
    n_head = job_attention_heads[job_idx],
    n_layers = job_layers[job_idx],
    
    n_simulation_sequences_train = job_n_sim_seqs[job_idx],
    n_human_sequences_train = job_n_human_seqs[job_idx],

    on_cluster = True,
    save_folder_name = 'Best_Param_Results',
    save_file_name = save_file_name)

{'model_name': 'LSTM', 'train_seq_part': 'fix_and_choice', 'n_simulation_sequences_train': 70432, 'n_human_sequences_train': 0, 'n_sequences_test': 500, 'n_sequences_final_performance': 500, 'd_model': 64, 'n_layers': 0, 'n_head': 0, 'sim_lr': 0.002, 'human_lr': 0, 'batch_size': 32, 'run_idx': 0, 'on_cluster': True, 'test_batch_increment_sim': 200, 'test_batch_increment_human': 200, 'save_folder_name': 'Best_Param_Results', 'save_file_name': 'run_0_model_name_LSTM_fix_and_sim_bp'}
<class 'numpy.int64'>
Loading Data
cpu
Training the model
Training on simulated data
number of simulation seqeuences: 6432number of human seqeuences: 0 sim test loss: 4.9539231061935425 human test loss 5.690425902605057
number of simulation seqeuences: 12832number of human seqeuences: 0 sim test loss: 4.938186913728714 human test loss 5.6557090282440186
number of simulation seqeuences: 19232number of human seqeuences: 0 sim test loss: 4.806150555610657 human test loss 5.699034601449966
number of simulation se

In [19]:
len(job_model_names)

240